# The Battle of Neighborhoods

In [1]:
import pandas as pd # Data Analysis
import numpy as np # Handles Data in a Vectorized Manner
import requests #handles requests
import folium # Plotting
from geopy.geocoders import Nominatim # address to Latitude and Longitude
from geopy.extra.rate_limiter import RateLimiter # reduces the "Too Many Requests Error"
from bs4 import BeautifulSoup # web scraping


print('Modules Imported!')

Modules Imported!


In [2]:
# Web Sources for scraping

NYC_url = 'https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm'
LA_url = 'http://www.laalmanac.com/communications/cm02_communities.php'
CHI_url = 'https://www.seechicagorealestate.com/chicago-zip-codes-by-neighborhood.php'

## Web Scraping and setting up DataFrames

New York City

In [3]:
source1 = requests.get(NYC_url).text
soup1 = BeautifulSoup(source1,'lxml')
table1 = soup1.find('table')

readme_html1 = pd.read_html(str(table1))
NYC_df = pd.DataFrame(readme_html1[0])
print(NYC_df.shape, '\n', NYC_df.head())

(42, 3) 
   Borough                Neighborhood                   ZIP Codes
0   Bronx               Central Bronx         10453, 10457, 10460
1   Bronx      Bronx Park and Fordham         10458, 10467, 10468
2   Bronx  High Bridge and Morrisania         10451, 10452, 10456
3   Bronx  Hunts Point and Mott Haven  10454, 10455, 10459, 10474
4   Bronx   Kingsbridge and Riverdale                10463, 10471


Los Angeles 

In [4]:
source2 = requests.get(LA_url).text
soup2 = BeautifulSoup(source2, 'lxml')
table2 = soup2.find('table')

readme_html2 = pd.read_html(str(table2))
LA_df = pd.DataFrame(readme_html2[0])
print(LA_df.shape, '\n', LA_df.head())

(643, 2) 
             City/Community   Zip Code(s)
0                    Acton         93510
1             Agoura Hills         91301
2  Agoura Hills (PO Boxes)         91376
3               Agua Dulce         91390
4                 Alhambra  91801, 91803


Chicago

In [5]:
source3 = requests.get(CHI_url).text
soup3 = BeautifulSoup(source3, 'lxml')
table3 = soup3.find('table')

readme_html3 = pd.read_html(str(table3))
CHI_df = pd.DataFrame(readme_html3[0])
print(CHI_df.shape, '\n', CHI_df.head())

(199, 2) 
                     0             1
0            Downtown      Zip Code
1  Cathedral District         60611
2     Central Station         60605
3       Dearborn Park         60605
4          Gold Coast  60610, 60611


Renaming columns to be consistent through each DataFrame

In [6]:
NYC_df.rename(columns={'ZIP Codes': 'zip_codes'}, inplace=True)
LA_df.rename(columns={'City/Community': 'Neighborhood', 'Zip Code(s)': 'zip_codes'}, inplace=True)
CHI_df.columns = ['Neighborhood','zip_codes']

print(list(NYC_df.columns))
print(list(LA_df.columns))
print(list(CHI_df.columns))

['Borough', 'Neighborhood', 'zip_codes']
['Neighborhood', 'zip_codes']
['Neighborhood', 'zip_codes']


## Cleaning DataFrames

In [7]:
#Checking NA and NULLs in each DataFrame

print(NYC_df.isna().sum())
print(NYC_df.isnull().sum())
print('\n')
print(LA_df.isna().sum())
print(LA_df.isnull().sum())
print('\n')
print(CHI_df.isna().sum())
print(CHI_df.isnull().sum())

Borough         0
Neighborhood    0
zip_codes       0
dtype: int64
Borough         0
Neighborhood    0
zip_codes       0
dtype: int64


Neighborhood    0
zip_codes       0
dtype: int64
Neighborhood    0
zip_codes       0
dtype: int64


Neighborhood    7
zip_codes       7
dtype: int64
Neighborhood    7
zip_codes       7
dtype: int64


In [8]:
#Investigating the CHI DataFrame

#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)
#CHI_df

In [9]:
CHI_df.dropna(inplace=True)
print(CHI_df.isna().sum())
print(CHI_df.isnull().sum())

Neighborhood    0
zip_codes       0
dtype: int64
Neighborhood    0
zip_codes       0
dtype: int64


### Exploding DataFrame by its ZIP Code

since there are multiple zip codes for each neighborhood, I need to separate them so the the geo locator can locate each zipcode

In [10]:
new_df1 = pd.DataFrame(NYC_df.zip_codes.str.split(',').tolist(), index= NYC_df.Neighborhood).stack()
new_df1 = new_df1.reset_index([0, 'Neighborhood'])
NYC_df2 = new_df1
NYC_df2.columns = ['Neighborhood','zip_codes']
NYC_df2

,Neighborhood,zip_codes
0,Central Bronx,10453
1,Central Bronx,10457
2,Central Bronx,10460
3,Bronx Park and Fordham,10458
4,Bronx Park and Fordham,10467
...,...,...
173,South Shore,10312
174,Stapleton and St. George,10301
175,Stapleton and St. George,10304
176,Stapleton and St. George,10305


In [11]:
new_df2 = pd.DataFrame(LA_df.zip_codes.str.split(',').tolist(), index= LA_df.Neighborhood).stack()
new_df2 = new_df2.reset_index([0, 'Neighborhood'])
LA_df2 = new_df2
LA_df2.columns = ['Neighborhood','zip_codes']
LA_df2

,Neighborhood,zip_codes
0,Acton,93510
1,Agoura Hills,91301
2,Agoura Hills (PO Boxes),91376
3,Agua Dulce,91390
4,Alhambra,91801
...,...,...
975,Woodland Hills (Los Angeles),91364
976,Woodland Hills (Los Angeles),91367
977,Woodland Hills (PO Boxes) (Los Angeles),91365
978,Woodland Hills (PO Boxes) (Los Angeles),91372


In [12]:
new_df3 = pd.DataFrame(CHI_df.zip_codes.str.split(',').tolist(), index= CHI_df.Neighborhood).stack()
new_df3 = new_df3.reset_index([0, 'Neighborhood'])
new_df3.drop(new_df3.index[0], inplace=True)
new_df3.reset_index(inplace=True)
CHI_df2 = new_df3
CHI_df2.columns = ['old index' , 'Neighborhood','zip_codes']
CHI_df2.drop(['old index'], axis=1, inplace= True)
CHI_df2

,Neighborhood,zip_codes
0,Cathedral District,60611
1,Central Station,60605
2,Dearborn Park,60605
3,Gold Coast,60610
4,Gold Coast,60611
...,...,...
355,Washington Park,60609
356,Washington Park,60615
357,Washington Park,60621
358,Washington Park,60637


## Finding Geo Coordinates for each ZIP Code

In [13]:
#geolocator = Nominatim(user_agent='google_agent')
#geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
#NYC_df2['location'] = NYC_df2['zip_codes'].apply(geocode)

#NYC_df2['point'] = NYC_df2['location'].apply(lambda loc: tuple(loc.point) if loc else None)

#print('DONE!')

In [14]:
#LA_df2['location'] = LA_df2['zip_codes'].apply(geocode)

#LA_df2['point'] = LA_df2['location'].apply(lambda loc: tuple(loc.point) if loc else None)

#print('DONE!')

In [15]:
#CHI_df2['location'] = CHI_df2['zip_codes'].apply(geocode)

#CHI_df2['point'] = CHI_df2['location'].apply(lambda loc: tuple(loc.point) if loc else None)

#print('DONE!')

Saving results so that I dont have to re run the code each time. Takes a while to find to reduce the "Too Many Requests Error"

In [16]:
#NYC_df2.to_csv('NYC_df2', index=False)
#LA_df2.to_csv('LA_df2', index=False)
#CHI_df2.to_csv('CHI_df2', index=False)

#print('DONE!')

In [17]:
NYC_df2 = pd.read_csv('NYC_df2')
LA_df2 = pd.read_csv('LA_df2')
CHI_df2 = pd.read_csv('CHI_df2')

Cleaning DataFrames Again. Because of there were coordinates and location outside of the US. I believe this happened because of the limitations of the Nominatim Function.

In [18]:
NYC_df2 = NYC_df2[NYC_df2.location.str.contains('New York')]
NYC_df2

,Neighborhood,zip_codes,location,point
0,Central Bronx,10453,"The Bronx, Bronx County, New York, 10453, Unit...","(40.85234385146179, -73.91196955537043, 0.0)"
1,Central Bronx,10457,"The Bronx, Bronx County, New York, 10457, Unit...","(40.84796417510888, -73.89770955547682, 0.0)"
3,Bronx Park and Fordham,10458,"The Bronx, Bronx County, New York, 10458, Unit...","(40.86156201215534, -73.88877720559567, 0.0)"
4,Bronx Park and Fordham,10467,"The Bronx, Bronx County, New York, 10467, Unit...","(40.87452493805371, -73.86794642958414, 0.0)"
5,Bronx Park and Fordham,10468,"The Bronx, Bronx County, New York, 10468, Unit...","(40.8710838, -73.8941052, 0.0)"
...,...,...,...,...
173,South Shore,10312,"Staten Island, Richmond County, New York, 1031...","(40.5436525, -74.16830055, 0.0)"
174,Stapleton and St. George,10301,"Staten Island, Richmond County, New York, 1030...","(40.63979875, -74.07574570091533, 0.0)"
175,Stapleton and St. George,10304,"Todt Hill, Staten Island, Richmond County, New...","(40.606031, -74.086625, 0.0)"
176,Stapleton and St. George,10305,"Todt Hill, Staten Island, Richmond County, New...","(40.609344, -74.076293, 0.0)"


In [19]:
CHI_df2 = CHI_df2[CHI_df2.location.str.contains('Chicago')]
CHI_df2

,Neighborhood,zip_codes,location,point
0,Cathedral District,60611,"Near North Side, Chicago, Cook County, Illinoi...","(41.89513283566048, -87.62281879221906, 0.0)"
1,Central Station,60605,"Near South Side, Chicago, Cook County, Illinoi...","(41.8658506, -87.6099423, 0.0)"
2,Dearborn Park,60605,"Near South Side, Chicago, Cook County, Illinoi...","(41.8658506, -87.6099423, 0.0)"
3,Gold Coast,60610,"Near North Side, Chicago, Cook County, Illinoi...","(41.8884984, -87.6292815, 0.0)"
4,Gold Coast,60611,"Near North Side, Chicago, Cook County, Illinoi...","(41.89513283566048, -87.62281879221906, 0.0)"
...,...,...,...,...
355,Washington Park,60609,"Bridgeport, Chicago, Cook County, Illinois, 60...","(41.8249226, -87.6381346, 0.0)"
356,Washington Park,60615,"Hyde Park, Chicago, Cook County, Illinois, 606...","(41.80110300957798, -87.59432360924957, 0.0)"
357,Washington Park,60621,"Englewood, Chicago, Cook County, Illinois, 606...","(41.7775939545224, -87.63663948181107, 0.0)"
358,Washington Park,60637,"Hyde Park, Chicago, Cook County, Illinois, 606...","(41.7884978, -87.5793199, 0.0)"


Due to NA/NULL values the same operation of filtering the data didnt work. So I had to drop them again. Possibly due to the limitations of Nominatim.


In [20]:
print(LA_df2.isna().sum())
print(LA_df2.isnull().sum())

Neighborhood     0
zip_codes        0
location        88
point           88
dtype: int64
Neighborhood     0
zip_codes        0
location        88
point           88
dtype: int64


In [21]:
LA_df2.dropna(inplace=True)
print(LA_df2.isna().sum())
print(LA_df2.isnull().sum())

Neighborhood    0
zip_codes       0
location        0
point           0
dtype: int64
Neighborhood    0
zip_codes       0
location        0
point           0
dtype: int64


In [22]:
LA_df2 = LA_df2[LA_df2.location.str.contains('Los Angeles')]
LA_df2


,Neighborhood,zip_codes,location,point
0,Acton,93510,"Los Angeles County, California, 93510, United ...","(34.46103834645381, -118.21138549680546, 0.0)"
1,Agoura Hills,91301,"Malibu Junction, Agoura Hills, Los Angeles Cou...","(34.15295555, -118.75932802244274, 0.0)"
5,Alhambra,91803,"Alhambra, Los Angeles County, California, 9180...","(34.07170388325506, -118.1458605938974, 0.0)"
7,Alhambra (PO Boxes),91802,"Alhambra, Los Angeles County, California, 9180...","(34.0927825, -118.1263605, 0.0)"
10,Altadena,91001,"Linda Vista, Altadena, Los Angeles County, Cal...","(34.1900323, -118.1325732, 0.0)"
...,...,...,...,...
956,Wilmington (Los Angeles),90744,"Thenard, Los Angeles, Los Angeles County, Cali...","(33.7933964, -118.2400379, 0.0)"
958,Wilshire Center (Los Angeles),90004,"Los Angeles, Los Angeles County, California, 9...","(34.071844381995426, -118.3023158073664, 0.0)"
966,Wilshire-La Brea (Los Angeles),90036,"Los Angeles, Los Angeles County, California, 9...","(34.06737818754923, -118.352266061733, 0.0)"
968,Windsor Hills (Los Angeles),90043,"Los Angeles, Los Angeles County, California, 9...","(33.992140810110676, -118.33125048227424, 0.0)"


In [23]:
NYC_df2.drop_duplicates(subset= 'zip_codes', keep= False, inplace=True)
LA_df2.drop_duplicates(subset= 'zip_codes', keep= False, inplace=True)
CHI_df2.drop_duplicates(subset= 'zip_codes', keep= False, inplace=True)

print('Duplicates Removed!')

Duplicates Removed!


Setting up for column split insteads of "point" it'll be  "Latitude" & "Longitude"

In [24]:
NYC_df2=NYC_df2.replace('\(','',regex=True).astype(str)
NYC_df2=NYC_df2.replace('\)','',regex=True).astype(str)
NYC_df2=NYC_df2.replace('\, 0.0','',regex=True).astype(str)

LA_df2=LA_df2.replace('\(','',regex=True).astype(str)
LA_df2=LA_df2.replace('\)','',regex=True).astype(str)
LA_df2=LA_df2.replace('\, 0.0','',regex=True).astype(str)

CHI_df2=CHI_df2.replace('\(','',regex=True).astype(str)
CHI_df2=CHI_df2.replace('\)','',regex=True).astype(str)
CHI_df2=CHI_df2.replace('\, 0.0','',regex=True).astype(str)

In [25]:
NYC_df2.reset_index(inplace=True)
LA_df2.reset_index(inplace=True)
CHI_df2.reset_index(inplace=True)

Made new DataFrame with Lat and Long values

In [26]:
NYC_df2.drop(['index'], axis=1, inplace= True)
LA_df2.drop(['index'], axis=1, inplace= True)
CHI_df2.drop(['index'], axis=1, inplace= True)

In [27]:
split_df1 = NYC_df2['point'].str.split(",")
data1 = split_df1.to_list()
names1 = ['Latitude', 'Longitude']
M1_df = pd.DataFrame(data1, columns=names1)

In [28]:
split_df2 = LA_df2['point'].str.split(",")
data2 = split_df2.to_list()
names2 = ['Latitude', 'Longitude']
M2_df = pd.DataFrame(data2, columns=names2)

In [29]:
split_df3 = CHI_df2['point'].str.split(",")
data3 = split_df3.to_list()
names3 = ['Latitude', 'Longitude']
M3_df = pd.DataFrame(data3, columns=names3)

In [30]:
NYC_df3 = pd.concat([NYC_df2, M1_df], axis=1, join='inner')
LA_df3 = pd.concat([LA_df2, M2_df], axis=1, join='inner')
CHI_df3 = pd.concat([CHI_df2, M3_df], axis=1, join='inner')

print('Concatenated objects')

Concatenated objects


In [31]:
NYC_df3.drop(['point'], axis=1, inplace= True)
LA_df3.drop(['point'], axis=1, inplace= True)
CHI_df3.drop(['point'], axis=1, inplace= True)

print("Columns Dropped!")

Columns Dropped!


In [32]:
convert_dict = {"zip_codes": int, "Latitude": float, "Longitude": float}
NYC_df3 = NYC_df3.astype(convert_dict)
LA_df3 = LA_df3.astype(convert_dict)
CHI_df3 = CHI_df3.astype(convert_dict)

In [33]:
print(NYC_df3.dtypes)
print('\n')
print(LA_df3.dtypes)
print('\n')
print(CHI_df3.dtypes)

Neighborhood     object
zip_codes         int64
location         object
Latitude        float64
Longitude       float64
dtype: object


Neighborhood     object
zip_codes         int64
location         object
Latitude        float64
Longitude       float64
dtype: object


Neighborhood     object
zip_codes         int64
location         object
Latitude        float64
Longitude       float64
dtype: object


In [34]:
address1 = 'New York City, NY'

geolocator1 = Nominatim(user_agent="explorer1")
location = geolocator1.geocode(address1)
latitude1 = location.latitude
longitude1 = location.longitude
print('The geograpical coordinate of New York City, NY are {}, {}.'.format(latitude1, longitude1))

The geograpical coordinate of New York City, NY are 40.7127281, -74.0060152.


In [35]:
address2 = 'Los Angeles, CA'

geolocator2 = Nominatim(user_agent="explorer2")
location = geolocator2.geocode(address2)
latitude2 = location.latitude
longitude2 = location.longitude
print('The geograpical coordinate of Los Angeles, CA are {}, {}.'.format(latitude2, longitude2))

The geograpical coordinate of Los Angeles, CA are 34.0536909, -118.2427666.


In [36]:
address3 = 'Chicago, IL'

geolocator3 = Nominatim(user_agent="explorer3")
location = geolocator3.geocode(address3)
latitude3 = location.latitude
longitude3 = location.longitude
print('The geograpical coordinate of Chicago, IL {}, {}.'.format(latitude3, longitude3))

The geograpical coordinate of Chicago, IL 41.8755616, -87.6244212.


In [37]:
map_NYC = folium.Map(location=[latitude1, longitude1], zoom_start=11)

for lat, lng, label in zip(NYC_df3['Latitude'], NYC_df3['Longitude'], NYC_df3['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NYC)  
    
map_NYC

In [38]:
map_LA = folium.Map(location=[latitude2, longitude2], zoom_start=11)

for lat, lng, label in zip(LA_df3['Latitude'], LA_df3['Longitude'], LA_df3['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_LA)  
    
map_LA

In [39]:
map_CHI = folium.Map(location=[latitude3, longitude3], zoom_start=11)

for lat, lng, label in zip(CHI_df3['Latitude'], CHI_df3['Longitude'], CHI_df3['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_CHI)  
    
map_CHI

In [40]:
CLIENT_ID = 'CCU2NDZIW42VAGVMA0ABT3MIBGES1AII4R5MG0DGWE3F55V4' # your Foursquare ID
CLIENT_SECRET = 'HFOEBHKSEDYXG2ANCMPQDEE1XSWC2I20T5M2GHM0GJM22YYY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [41]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION,
    latitude1, 
    longitude1, 
    radius,
    LIMIT,
    "52e81612bcbc57f1066b79ff")

In [42]:
url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION,
    latitude2, 
    longitude2, 
    radius,
    LIMIT,
    "52e81612bcbc57f1066b79ff")

In [43]:
url3 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION,
    latitude3, 
    longitude3, 
    radius,
    LIMIT,
    "52e81612bcbc57f1066b79ff")

In [44]:
results1 = requests.get(url1).json()
results2 = requests.get(url2).json()
results3 = requests.get(url3).json()

print("Got Results")

Got Results


In [45]:
results1

{'meta': {'code': 200, 'requestId': '5e578fb0b57e88001b77b1a4'},
 'response': {'headerLocation': 'Downtown Manhattan',
  'headerFullLocation': 'Downtown Manhattan, New York',
  'headerLocationGranularity': 'neighborhood',
  'query': 'halal',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 40.7172281045, 'lng': -74.00008952063419},
   'sw': {'lat': 40.7082280955, 'lng': -74.0119408793658}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53aaf72a498e19106dd204aa',
       'name': 'Halal food truck',
       'location': {'address': 'Ann Street',
        'crossStreet': 'Nassau Street',
        'lat': 40.71040239758247,
        'lng': -74.00705705666762,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.71040239758247,
          'lng': -74.0070570566

In [46]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [47]:
# venues = results1['response']['groups'][0]['items']
#     
# nearby_venues = pd.json_normalize(venues) # flatten JSON
# 
# # filter columns
# filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
# nearby_venues =nearby_venues.loc[:, filtered_columns]
# 
# # filter the category for each row
# nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# 
# # clean columns
# nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
# 
# nearby_venues.head()

In [48]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            latitude3, 
            longitude3, 
            radius,
            LIMIT,
            "52e81612bcbc57f1066b79ff") #HALAL Resturant ID
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [49]:
LA_venues = getNearbyVenues(names = LA_df3['zip_codes'],
                             latitudes = LA_df3['Latitude'],
                             longitudes = LA_df3['Longitude'])


93510
91803
91802
91001
90044
91706
90270
91307
90706
90210
90211
90212
90063
91010
91502
91505
91506
90745
90746
90747
91384
91383
91724
91311
93551
90063
91711
90040
91722
91723
91724
90230
90232
91765
90810
90242
90013
90021
90021
90029
91732
91733
90245
91387
90247
90249
91206
91740
91741
91344
91745
90710
90716
90254
91744
91748
91715
90303
90304
90305
90306
91706
93553
90006
90631
90638
90056
93550
90712
90713
90715
93534
93536
90260
90018
90304
90717
90802
90806
90808
90813
90814
90815
90037
90062
90262
90265
90266
90267
90270
91016
91754
91755
93550
93552
90275
90723
91103
91105
91106
91107
91188
91109
91124
91184
93553
93563
90660
91767
91768
91770
91748
91773
91775
91776
91778
91350
91351
91354
90670
90404
90401
91024
90755
91733
90280
90062
90003
91030
91381
90290
90503
90504
90505
91384
93563
91403
90058
90073
91789
91790
91791
91792
90048
90602
90603
90604
90608
90059


In [50]:
LA_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,93510,34.461038,-118.211385,Max's Take Out,41.879570,-87.626694,Hot Dog Joint
1,93510,34.461038,-118.211385,Epic Burger,41.875002,-87.627525,Burger Joint
2,93510,34.461038,-118.211385,Twisted Tacos,41.879352,-87.626624,Halal Restaurant
3,91803,34.071704,-118.145861,Max's Take Out,41.879570,-87.626694,Hot Dog Joint
4,91803,34.071704,-118.145861,Epic Burger,41.875002,-87.627525,Burger Joint


In [51]:
CHI_venues = getNearbyVenues(names = CHI_df3['zip_codes'],
                             latitudes = CHI_df3['Latitude'],
                             longitudes = CHI_df3['Longitude'])

60602
60603
60604
60605
60606
60606
60612
60614
60618
60641
60659
60625
60639
60634
60124
60302
60620
60643
60616
60623
60621
60636
60624
60643
60653
60619
60649
60615
60621


In [52]:
CHI_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,60602,41.882985,-87.629238,Max's Take Out,41.879570,-87.626694,Hot Dog Joint
1,60602,41.882985,-87.629238,Epic Burger,41.875002,-87.627525,Burger Joint
2,60602,41.882985,-87.629238,Twisted Tacos,41.879352,-87.626624,Halal Restaurant
3,60603,41.881228,-87.628342,Max's Take Out,41.879570,-87.626694,Hot Dog Joint
4,60603,41.881228,-87.628342,Epic Burger,41.875002,-87.627525,Burger Joint


In [53]:
NYC_venues = getNearbyVenues(names = NYC_df3['zip_codes'],
                             latitudes = NYC_df3['Latitude'],
                             longitudes = NYC_df3['Longitude'], 
                             radius = 500)

10453
10457
10458
10467
10468
10451
10452
10456
10454
10455
10459
10474
10463
10471
10466
10469
10470
10475
10461
10462
10464
10465
10472
10473
11213
11216
11233
11209
11214
11228
11204
11218
11219
11230
11236
11239
11223
11224
11229
11235
11201
11205
11215
11217
11231
11203
11210
11225
11207
11208
11211
11222
11220
11232
11206
11221
11237
10026
10027
10037
10039
10011
10019
10036
10029
10035
10022
10038
10280
10003
10021
10028
10065
10075
10023
10024
10025
10033
10034
10040
11361
11362
11364
11354
11355
11356
11357
11359
11360
11365
11366
11367
11412
11423
11432
11433
11435
11436
11102
11103
11104
11374
11375
11379
11385
11691
11692
11694
11697
11004
11413
11422
11426
11427
11428
11414
11415
11416
11417
11418
11420
11421
11368
11369
11372
11377
11378
10303
10306
10307
10308
10309
10312
10301
10304
10305
10314


In [54]:
NYC_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,10453,40.852344,-73.91197,Max's Take Out,41.879570,-87.626694,Hot Dog Joint
1,10453,40.852344,-73.91197,Epic Burger,41.875002,-87.627525,Burger Joint
2,10453,40.852344,-73.91197,Twisted Tacos,41.879352,-87.626624,Halal Restaurant
3,10457,40.847964,-73.89771,Max's Take Out,41.879570,-87.626694,Hot Dog Joint
4,10457,40.847964,-73.89771,Epic Burger,41.875002,-87.627525,Burger Joint


In [56]:
# NYC_venues.to_csv('NYC_venues', index=False)
# LA_venues.to_csv('LA_venues', index=False)
# CHI_venues.to_csv('CHI_venues', index=False)